In [1]:
!pip install tfx

Tahapan pengembangan dan validasi model ini terdiri dari beberapa proses seperti berikut.

- Proses pengembangan model. Pada proses ini menggunakan komponen Trainer.
- Proses analisis dan validasi model. Proses ini akan dibuat menggunakan komponen Resolver dan Evaluator.

# Membuat Tahapan Pengembangan Model

- **Fungsi transformed_name()**

  Fungsi ini digunakan untuk mengubah nama fitur yang telah melalui proses transform.

In [2]:
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

- **Fungsi gzip_reader_fn()**

  Ia merupakan fungsi yang digunakan untuk memuat data dalam format TFRecord.

In [3]:
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

- **Fungsi input_fn()**

  Ia digunakan untuk memuat transformed_feature yang dihasilkan oleh komponen Transform dan membaginya ke dalam beberapa batch.

In [4]:
import tensorflow as tf

In [5]:
def input_fn(file_pattern,
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset

- **Fungsi model_builder()**

  Fungsi inilah yang bertanggung jawab dalam membuat arsitektur model. Pada latihan ini, kita menggunakan salah satu embedding layer yang tersedia dan dapat diunduh melalui TensorFlow Hub

  https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/tensorFlow2/variations/universal-sentence-encoder/versions/2?tfhub-redirect=true

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [11]:
embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])

print(embeddings)

tf.Tensor(
[[-0.03133017 -0.06338634 -0.01607501 ... -0.03242778 -0.0457574
   0.05370456]
 [ 0.0508086  -0.01652434  0.01573779 ...  0.00976657  0.03170121
   0.01788118]], shape=(2, 512), dtype=float32)


In [15]:
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)

embedding_layer = hub.KerasLayer(
    "https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
    input_shape=[],  # Input shape is a single sentence
    dtype=tf.string,
    trainable=False  # Set to True if you want to fine-tune the embedding layer
)

embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]

    )

    print(model)
    model.summary()
    return model

- **Fungsi _get_serve_tf_examples_fn()**

  Fungsi ini digunakan untuk menjalankan tahapan preprocessing data pada raw request data.

In [17]:
def _get_serve_tf_examples_fn(model, tf_transform_output):

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):

        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

- **Fungsi run_fn()**

  Ia merupakan fungsi yang bertanggung jawab untuk menjalankan proses training model sesuai dengan parameter training yang diberikan.

In [20]:
from typing import NamedTuple
import os

class FnArgs(NamedTuple):
    serving_model_dir: str
    transform_graph_path: str
    train_files: str
    eval_files: str

In [21]:
def run_fn(fn_args: FnArgs) -> None:

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)


    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])

    # Build the model
    model = model_builder()


    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000,
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Seluruh fungsi di atas, akan disatukan ke dalam sebuah module file. Namun, sebelum membuat module file, kita perlu mendefinisikan nama dari module tersebut.

In [34]:
TRAINER_MODULE_FILE = "sarcasm_trainer.py"

Selanjutnya, buatlah sebuah module dengan kode berikut.

In [35]:
# writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "is_sarcastic"
FEATURE_KEY = "headline"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern,
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset

# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)


embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)


    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]

    )

    # print(model)
    model.summary()
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):

        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)


    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])

    # Build the model
    model = model_builder()


    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000,
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Kode di atas akan menghasilkan sebuah module dengan nama **sarcasm_trainer.py**. Module tersebut mengandung training function beserta beberapa helper function yang telah disebutkan sebelumnya.

Tahap selanjutnya adalah mendefinisikan komponen **Trainer()**. Komponen ini akan menerima beberapa input seperti berikut.

- **module_file** untuk menerima module file yang mengandung mengandung training function beserta beberapa helper function.
- **examples** untuk menerima dataset dari komponen ExampleGen.
- **schema** untuk menerima data schema dari komponen SchemaGen.
- **tansform_graph** untuk menerima transform graph yang dihasilkan dari komponen Transform.
- **train_args** untuk menampung parameter training.
- **eval_args** untuk menampung parameter testing atau evaluation.

In [36]:
import os
from tfx.proto import trainer_pb2
from tfx.components.trainer.component import Trainer

In [37]:
# from tfx.proto import trainer_pb2

# trainer  = Trainer(
#     module_file=os.path.abspath(TRAINER_MODULE_FILE),
#     examples = transform.outputs['transformed_examples'],
#     transform_graph=transform.outputs['transform_graph'],
#     schema=schema_gen.outputs['schema'],
#     train_args=trainer_pb2.TrainArgs(splits=['train']),
#     eval_args=trainer_pb2.EvalArgs(splits=['eval'])
# )
# interactive_context.run(trainer)

# Membuat Tahapan Analisis dan Validasi Model

- **Komponen Resolver**

  Untuk melakukan analisis dan validasi model, kita perlu menyediakan sebuah baseline model. Hal ini sangat penting terutama ketika kita memiliki lebih dari satu versi model dan ingin membandingkan dua buah versi model yang berbeda. Untuk melakukannya, kita bisa memanfaatkan komponen **Resolver()**.

In [38]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context = InteractiveContext()
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 1
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

- **Komponen Evaluator**

  Setelah mendefinisikan komponen Resolver, tahap selanjutnya adalah membuat beberapa konfigurasi untuk mengevaluasi model. Konfigurasi ini dibuat menggunakan library TFMA seperti berikut.

In [39]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='is_sarcastic')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[

            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]

)

Pada contoh kode di atas, kita mengatur beberapa konfigurasi seperti metrik untuk mengevaluasi model beserta nilai threshold dari suatu metrik. Selain menentukan metrik, kita juga dapat mengatur pembagian kelompok data berdasarkan fitur tertentu pada parameter slicing_specs. Namun, pada latihan ini, kita tidak membagi data ke dalam kelompok data tertentu karena kasus yang diangkat belum terlalu kompleks.


Setelah membuat konfigurasi yang akan digunakan untuk mengevaluasi model, tahap berikutnya adalah mendefinisikan komponen Evaluator. Komponen ini akan menerima beberapa input seperti berikut.
- examples untuk menerima dataset dari komponen ExampleGen.

- model untuk menerima model yang dihasilkan dari komponen Trainer.

- baseline_model untuk menampung baseline model yang disediakan oleh komponen Resolver.

- eval_config untuk menerima konfigurasi untuk mengevaluasi model.

In [51]:
# from tfx.components import Evaluator
# evaluator = Evaluator(
#     examples=example_gen.outputs['examples'],
#     model=trainer.outputs['model'],
#     baseline_model=model_resolver.outputs['model'],
#     eval_config=eval_config)

# interactive_context.run(evaluator)

Hasil evaluasi dari komponen ini dapat Anda visualisasikan menggunakan library TFMA seperti contoh kode di bawah ini.

In [49]:
# # Visualize the evaluation results
# eval_result = evaluator.outputs['evaluation'].get()[0].uri
# tfma_result = tfma.load_eval_result(eval_result)
# tfma.view.render_slicing_metrics(tfma_result)
# tfma.addons.fairness.view.widget_view.render_fairness_indicator(
#     tfma_result
# )

 # **Tools Lanjutan dalam Pengembangan dan Validasi Model**

  # Penjelasan Lanjutan Komponen Tuner

Pada latihan sebelumnya, kita telah melakukan tahap pengembangan model menggunakan komponen Trainer. Namun, pada latihan tersebut, kita hanya melakukan proses model training dan belum mencakup proses model tuning.

Pada TFX pipeline, proses model tuning akan dikerjakan oleh komponen Tuner. Ia akan melakukan proses model tuning secara otomatis dengan bantuan keras tuner. Seperti yang kita bahas pada modul sebelumnya, komponen ini menerima beberapa input seperti berikut.

- Data schema yang diperoleh dari tahap data validation.
- Transformed data yang diperoleh dari tahap data preprocessing.
- Parameter tuning dan training.
- Sebuah module file yang berisi *tuner function*.

In [52]:
def tuner_fn(fn_args):
  """Build the tuner using the KerasTuner API.
  Args:
    fn_args: Holds args used to tune models as name/value pairs.

  Returns:
    A namedtuple contains the following:
      - tuner: A BaseTuner that will be used for tuning.
      - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                    model , e.g., the training and validation dataset. Required
                    args depend on the above tuner's implementation.
  """
  # Memuat training dan validation dataset yang telah di-preprocessing
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
  train_set = input_fn(fn_args.train_files[0], tf_transform_output)
  val_set = input_fn(fn_args.eval_files[0], tf_transform_output)

  # Mendefinisikan strategi hyperparameter tuning
  tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory=fn_args.working_dir,
                     project_name='kt_hyperband')

  return TunerFnResult(
      tuner=tuner,
      fit_kwargs={
          "callbacks":[stop_early],
          'x': train_set,
          'validation_data': val_set,
          'steps_per_epoch': fn_args.train_steps,
          'validation_steps': fn_args.eval_steps
      }
  )

Pada contoh kode di atas, fungsi **tuner_fn()** menerima sebuah argumen **fn_args**. Argumen ini berisi segala hal yang dibutuhkan untuk menjalankan proses hyperparameter tuning. Sama halnya dengan fungsi run_fn(), fungsi ini juga membutuhkan *helper function*, seperti input_fn() dan model_builder().

Setelah membuat module file yang berisi tuner function, tahap selanjutnya adalah mendefinisikan komponen Tuner dan menjalankannya menggunakan **interactive_context()**

In [55]:
from tfx.components import Tuner
from tfx.proto import trainer_pb2

# tuner = Tuner(
#     module_file=tuner_module_file,
#     examples=transform.outputs['transformed_examples'],
#     transform_graph=transform.outputs['transform_graph'],
#     schema=schema_gen.outputs['schema'],
#     train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
#     eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
#     )

# interactive_context(tuner)

**Catatan:**

  Kedua kode di atas hanya digunakan sebagai contoh penerapan komponen Tuner dan tidak untuk dijalankan. Anda perlu menghubungkan tuner_fn() dengan module file yang berisi training function. Selain itu, Anda juga perlu menyesuaikan beberapa kode pada **model_builder()** untuk mendukung proses hyperparameter tuning seperti halnya ketika Anda menggunakan Keras Tuner.